# InfluxDB2.0 Primer

This notebook shows you how to write a dataframe into an Influx instance. Keep all your secret vars in a locally stored `.env` file

In [33]:
import pandas as pd
import os
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from dotenv import find_dotenv, load_dotenv
# You can generate a Token from the "Tokens Tab" in the UI
load_dotenv(find_dotenv())



True

Choose some city from a list

In [39]:
print(df['City'].unique())

['Jieyang' 'Beijing' 'Kunming' 'Hangzhou' 'Chongqing' 'Qingdao' 'Haikou'
 'Qiqihar' 'Guiyang' 'Ürümqi' 'Shenzhen' 'Yunfu' 'Xuchang' 'Yinchuan'
 'Shenyang' 'Lhasa' 'Shanghai' 'Changchun' 'Foshan' 'Nanning' 'Fushun'
 'Hefei' 'Chengdu' 'Hohhot' 'Qinhuangdao' 'Shijiazhuang' 'Shantou'
 'Zhengzhou' 'Nanjing' 'Xining' 'Xi’an' 'Zhuzhou' 'Wuhan' 'Tianjin'
 'Changzhou' 'Nanchang' 'Shiyan' 'Harbin' 'Xinxiang' 'Suzhou' 'Lanzhou'
 'Jinan' 'Changsha' 'Hegang' 'Anyang' 'Taiyuan' 'Guangzhou' 'Fuzhou'
 'Wuxi' 'Ningbo' 'Xiamen' 'Dongguan' 'Hamilton' 'Calgary' 'Winnipeg'
 'Halifax' 'Kitchener' 'Edmonton' 'Mississauga' 'Surrey' 'Québec'
 'Vancouver' 'Victoria' 'Montréal' 'Toronto' 'Ottawa' 'London']


# Set up InfluxDB bindings 

Documentation on Python bindings with examples can be found here:
https://github.com/influxdata/influxdb-client-python

In [34]:
influx = InfluxDBClient(url=os.environ['INFLUX_HOST'], token=os.environ['INFLUX_TOKEN'])
write_api = influx.write_api(write_options=SYNCHRONOUS)


In [35]:
df = pd.read_csv('https://raw.githubusercontent.com/AntonBiryukovUofC/air_quality_check/luis-testing/src/data/waqi-covid19-airqualitydata-filtered.csv',
                 parse_dates = ['Date']).set_index('Date')


In [36]:
cols = ['no2','o3','co','so2','temperature','City','Country']
df_to_write = df.loc[df['City'] == 'Ottawa',cols]
df_to_write.sample(10)

,no2,o3,co,so2,temperature,City,Country
Date,,,,,,,
2017-04-11,4.7,12.8,0.3,NaN,NaN,Ottawa,CA
2020-02-09,12.6,16.5,0.2,0.3,NaN,Ottawa,CA
2018-06-04,2.8,15.0,0.2,NaN,13.6,Ottawa,CA
2019-06-29,1.9,23.3,0.2,0.2,19.4,Ottawa,CA
2016-06-06,2.8,24.8,0.2,NaN,NaN,Ottawa,CA
2020-08-02,3.0,14.3,0.2,NaN,20.9,Ottawa,CA
2016-06-23,3.8,15.0,0.2,NaN,NaN,Ottawa,CA
2017-06-17,2.8,14.5,0.2,NaN,NaN,Ottawa,CA
2017-01-03,7.5,16.8,0.2,1.5,NaN,Ottawa,CA


In [37]:
write_api.write(os.environ['INFLUX_BUCKET'], org, record=df_to_write, data_frame_measurement_name='airquality',data_frame_tag_columns = ['City','Country'])

In [38]:
df_to_write.shape

(10628, 7)

array(['Jieyang', 'Beijing', 'Kunming', 'Hangzhou', 'Chongqing',
       'Qingdao', 'Haikou', 'Qiqihar', 'Guiyang', 'Ürümqi', 'Shenzhen',
       'Yunfu', 'Xuchang', 'Yinchuan', 'Shenyang', 'Lhasa', 'Shanghai',
       'Changchun', 'Foshan', 'Nanning', 'Fushun', 'Hefei', 'Chengdu',
       'Hohhot', 'Qinhuangdao', 'Shijiazhuang', 'Shantou', 'Zhengzhou',
       'Nanjing', 'Xining', 'Xi’an', 'Zhuzhou', 'Wuhan', 'Tianjin',
       'Changzhou', 'Nanchang', 'Shiyan', 'Harbin', 'Xinxiang', 'Suzhou',
       'Lanzhou', 'Jinan', 'Changsha', 'Hegang', 'Anyang', 'Taiyuan',
       'Guangzhou', 'Fuzhou', 'Wuxi', 'Ningbo', 'Xiamen', 'Dongguan',
       'Hamilton', 'Calgary', 'Winnipeg', 'Halifax', 'Kitchener',
       'Edmonton', 'Mississauga', 'Surrey', 'Québec', 'Vancouver',
       'Victoria', 'Montréal', 'Toronto', 'Ottawa', 'London'],
      dtype=object)